In [1]:
#import packages
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [25]:
#inherit openai gym environment class by passing env into the class

class solarEnv(Env):
    def __init__(self):
        #initialize the action, space 
        self.action_space = Discrete(3) #the actions: STORE, CONSUME SELL
        self.observation_space = Box(low=np.array([0]), high = np.array([100])) #the energy level, lowest is 0 and highest is 100
        #starting energy level
        self.state = 0+random.randint(-10,10)
        #set operation duration
        self.operation_length = 60
    def step(self, action):
        #what do we do everytime we step
        #actions:
        #0 -1 = sell
        #1-1 = consume
        #2-1 -1 store
        self.state += action -1
        #reduce the operation length time by 1 second
        self.operation_length -=1
        
        #calculate the reward
        #if the energy level goes out of the optimal range, the agent is punished
        if self.state>=4 and self.state<=7:
            reward =1
        else:
            reward = -1
        #check if the operation is done
        if self.operation_length <=0:
            done = True
        else:
            done = False
        #set an information placeholder
        info ={}
        
        #return step information
        return self.state, reward, done, info
        
    def reset(self):#reset after every run
        #reset operation level
        self.state= 0 + random.randint(-10,10)
        self.operation_length=60
        

In [26]:
#set the environment to the showerenvironment
env = solarEnv()

In [27]:
env.observation_space.sample()

array([98.380684], dtype=float32)

In [29]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-52
Episode:2 Score:-50
Episode:3 Score:-46
Episode:4 Score:-60
Episode:5 Score:-32
Episode:6 Score:-60
Episode:7 Score:-60
Episode:8 Score:32
Episode:9 Score:-60
Episode:10 Score:-60


# create deep learning model with keras

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [7]:
states = env.observation_space.shape #the shape of our state
actions = env.action_space.n # the number of action we got

In [8]:
actions

3

In [9]:
def build_model(states, actions): #?
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [10]:
model = build_model(states, actions)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# Build agent with keras-rl